Photodiode Analysis Code
Ansley Kunnath
Updated 04/02/24
Run with Python 3.9.12

In [23]:
import mne
import numpy as np
from mne.preprocessing import (ICA)
from scipy.signal import find_peaks
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("TkAgg")
import scipy.stats as stats
print('matplotlib: {}'.format(matplotlib.__version__))


eeg_path = "/Users/andrewkim/Desktop/Photodiode/"  # You will need to change this location
file_name = "Subject_Example_2024-03-29_10-51-49"
file_eeg = eeg_path + file_name + ".eeg"
file_vhdr = eeg_path + file_name + ".vhdr"
file_vmrk = eeg_path + file_name + ".vmrk"

# Load and plot the raw data
raw = mne.io.read_raw_brainvision(file_vhdr)
events, event_id = mne.events_from_annotations(raw)

#raw.crop(tmin=22, tmax=190)
#raw.plot()


matplotlib: 3.8.3
Extracting parameters from /Users/andrewkim/Desktop/Photodiode/Subject_Example_2024-03-29_10-51-49.vhdr...
Setting channel info structure...
Used Annotations descriptions: ['Marker/Impedance', 'New Segment/', 'Stimulus/s1', 'Stimulus/s2', 'Stimulus/s3', 'Stimulus/s5']


/var/folders/m2/8_k5938s5_l8bzxcjsy3qyk00000gn/T/ipykernel_26556/1859356157.py:19: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_brainvision(file_vhdr)


In [24]:
# Create epochs for checkerboard events

stimulus_s1_events = events[events[:, 2] == event_id['Stimulus/s1']]
stimulus_s1_events, event_id
tmin, tmax = 0, 0.250  # start and end time around each event in seconds
epochs = mne.Epochs(raw, events=stimulus_s1_events, event_id=event_id['Stimulus/s1'],
                    tmin=tmin, tmax=tmax, baseline=None, preload=True)
epochs = epochs.pick_channels(['BIP3'])

#mean_amplitude = np.mean(epochs)
#print(mean_amplitude)


Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated
Loading data for 100 events and 126 original time points ...
0 bad epochs dropped
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


In [31]:
# Calculate the FIRST latency <10% average for each epoch --> 5.22 ms

    #Calculate the threshold using median absoluted deviation
        #mad_channel1 = scipy.stats.median_abs_deviation(df["Channel1"])
        #mad_channel2 = scipy.stats.median_abs_deviation(df["Channel2"])
        # print(max(mad_channel1), median(mad_channel1, min(mad_channel1)))

        # Find the first exceedance index for each channel
        # exceedance1 = df.index[df['Channel1'] > peak_height_threshold_channel1]
        # exceedance2 = df.index[df['Channel2'] > peak_height_threshold_channel2]
        #exceedance1 = df.index[df['Channel1'] > numpy.median(df['Channel1']) + mad_channel1 * mad_threshold1]
        #exceedance2 = df.index[df['Channel2'] > numpy.median(df['Channel2']) + mad_channel2 * mad_threshold2]

#absolute value of the photodiode data and the median is 

peak_latencies = []
for epoch in epochs.get_data():
    median_amplitude = np.median(epoch)
    median_amplitude = abs(median_amplitude)
    epoch = abs(epoch)
    epochdifference = median_amplitude - epoch
    screen_changes = np.argmax(epochdifference > ((0.9 * median_amplitude) + median_amplitude))
    print(screen_changes)
    if (epochdifference > ((0.9 * median_amplitude) + median_amplitude)).any():
        first_time = epochs.times[screen_changes]
    else:
        first_time = None
    peak_latencies.append(first_time)

# Calculate latencies in milliseconds
peak_latencies_ms = np.array(peak_latencies) * 1000
average_latency_ms = np.median(peak_latencies_ms)
total_events = len(peak_latencies_ms)

# Calculate latency CI in milliseconds
sem_latency = stats.sem(peak_latencies)  # SEM = std / sqrt(n)
confidence_level = 0.95
ci_width = sem_latency * stats.t.ppf((1 + confidence_level) / 2, len(peak_latencies_ms) - 1)
confidence_interval = (average_latency_ms - ci_width, average_latency_ms + ci_width)

print(f"Median Latency: {average_latency_ms:.3f}")
print(f"95% Confidence Interval for the Latencies: {confidence_interval}")
print(f"Total Events: {total_events}")


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


/var/folders/m2/8_k5938s5_l8bzxcjsy3qyk00000gn/T/ipykernel_26556/532967263.py:17: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  for epoch in epochs.get_data():


TypeError: unsupported operand type(s) for *: 'NoneType' and 'int'

In [27]:
# Calculate the FIRST latency <40% average for each epoch --> 5.0 ms

peak_latencies = []
for epoch in epochs.get_data():
    median_amplitude = np.median(epoch)
    screen_changes = np.argmax(epoch > (0.9 * median_amplitude))
    epoch = abs(epoch)
    epoch = epoch - median_amplitude
    median_amplitude = abs(median_amplitude)
    if (epoch > (0.6 * median_amplitude)).any():
        first_time = epochs.times[screen_changes]
    else:
        first_time = None
    peak_latencies.append(first_time)

# Calculate latencies in milliseconds
peak_latencies_ms = np.array(peak_latencies) * 1000
average_latency_ms = np.median(peak_latencies_ms)
total_events = len(peak_latencies_ms)

# Calculate latency CI in milliseconds
sem_latency = stats.sem(peak_latencies)  # SEM = std / sqrt(n)
confidence_level = 0.95
ci_width = sem_latency * stats.t.ppf((1 + confidence_level) / 2, len(peak_latencies_ms) - 1)
confidence_interval = (average_latency_ms - ci_width, average_latency_ms + ci_width)

print(f"Median Latency: {average_latency_ms}")
print(f"95% Confidence Interval for the Latencies: {confidence_interval}")
print(f"Total Events: {total_events}")


Median Latency: 2.0
95% Confidence Interval for the Latencies: (1.9976107396542528, 2.002389260345747)
Total Events: 100


/var/folders/m2/8_k5938s5_l8bzxcjsy3qyk00000gn/T/ipykernel_26556/219650503.py:4: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  for epoch in epochs.get_data():


In [8]:
# Histogram of latency distributions

plt.figure(figsize=(10, 6))
plt.hist(peak_latencies_ms, bins=10, color='skyblue', edgecolor='black')
plt.axvline(average_latency_ms, color='red', linestyle='dashed', linewidth=2)
plt.title('Distribution of Latencies')
plt.xlabel('Latency (milliseconds)')
plt.ylabel('Frequency')
plt.legend(['Average Latency', 'Latencies'])
plt.grid(True)

plt.show()

1   HIToolbox                           0x00000001b91245c8 _ZN15MenuBarInstance22EnsureAutoShowObserverEv + 120
2   HIToolbox                           0x00000001b9124188 _ZN15MenuBarInstance14EnableAutoShowEv + 60
3   HIToolbox                           0x00000001b90c7310 SetMenuBarObscured + 372
4   HIToolbox                           0x00000001b90c6ee8 _ZN13HIApplication15HandleActivatedEP14OpaqueEventRefhP15OpaqueWindowPtrh + 172
5   HIToolbox                           0x00000001b90c0fcc _ZN13HIApplication13EventObserverEjP14OpaqueEventRefPv + 296
6   HIToolbox                           0x00000001b9087cd0 _NotifyEventLoopObservers + 176
7   HIToolbox                           0x00000001b90c096c AcquireEventFromQueue + 432
8   HIToolbox                           0x00000001b90afc84 ReceiveNextEventCommon + 320
9   HIToolbox                           0x00000001b90afb2c _BlockUntilNextEventMatchingListInModeWithFilter + 72
10  AppKit                              0x00000001b2c5c424 _DPS

1   HIToolbox                           0x00000001b90af90c _ZN15MenuBarInstance22RemoveAutoShowObserverEv + 44
2   HIToolbox                           0x00000001b90c7214 SetMenuBarObscured + 120
3   HIToolbox                           0x00000001b90ce390 _ZN13HIApplication11FrontUILostEv + 44
4   HIToolbox                           0x00000001b90c704c _ZN13HIApplication15HandleActivatedEP14OpaqueEventRefhP15OpaqueWindowPtrh + 528
5   HIToolbox                           0x00000001b90c0f70 _ZN13HIApplication13EventObserverEjP14OpaqueEventRefPv + 204
6   HIToolbox                           0x00000001b9087cd0 _NotifyEventLoopObservers + 176
7   HIToolbox                           0x00000001b90c096c AcquireEventFromQueue + 432
8   HIToolbox                           0x00000001b90afc84 ReceiveNextEventCommon + 320
9   HIToolbox                           0x00000001b90afb2c _BlockUntilNextEventMatchingListInModeWithFilter + 72
10  AppKit                              0x00000001b2c5c424 _DPSNextEv

In [8]:
import numpy as np
import matplotlib.pyplot as plt

# Scatter plot of the latencies
plt.figure(figsize=(10, 6))
plt.scatter(range(len(peak_latencies_ms)), peak_latencies_ms, color='black')
plt.axhline(average_latency_ms, color='red', linestyle='dashed', linewidth=2)
plt.title('Latencies')
plt.xlabel('Event')
plt.ylabel('Latency (milliseconds)')
plt.grid(True)

plt.show()

1   HIToolbox                           0x00000001b91245c8 _ZN15MenuBarInstance22EnsureAutoShowObserverEv + 120
2   HIToolbox                           0x00000001b90ee478 _ZL17BroadcastInternaljPvh + 184
3   SkyLight                            0x00000001b46044dc _ZN12_GLOBAL__N_123notify_datagram_handlerEj15CGSDatagramTypePvmS1_ + 896
4   SkyLight                            0x00000001b49304c4 _ZN21CGSDatagramReadStream26dispatchMainQueueDatagramsEv + 228
5   SkyLight                            0x00000001b49303c0 ___ZN21CGSDatagramReadStream15mainQueueWakeupEv_block_invoke + 28
6   libdispatch.dylib                   0x00000001af7749dc _dispatch_call_block_and_release + 32
7   libdispatch.dylib                   0x00000001af776504 _dispatch_client_callout + 20
8   libdispatch.dylib                   0x00000001af784d1c _dispatch_main_queue_drain + 928
9   libdispatch.dylib                   0x00000001af78496c _dispatch_main_queue_callback_4CF + 44
10  CoreFoundation                      

KeyboardInterrupt: 

1   HIToolbox                           0x00000001b91245c8 _ZN15MenuBarInstance22EnsureAutoShowObserverEv + 120
2   HIToolbox                           0x00000001b90c727c SetMenuBarObscured + 224
3   HIToolbox                           0x00000001b90c6ee8 _ZN13HIApplication15HandleActivatedEP14OpaqueEventRefhP15OpaqueWindowPtrh + 172
4   HIToolbox                           0x00000001b90c0fcc _ZN13HIApplication13EventObserverEjP14OpaqueEventRefPv + 296
5   HIToolbox                           0x00000001b9087cd0 _NotifyEventLoopObservers + 176
6   AppKit                              0x00000001b2dff5b0 -[NSWindow(NSEventRouting) _handleMouseDownEvent:isDelayedEvent:] + 2428
7   AppKit                              0x00000001b2d89ab8 -[NSWindow(NSEventRouting) _reallySendEvent:isDelayedEvent:] + 1984
8   AppKit                              0x00000001b2d89124 -[NSWindow(NSEventRouting) sendEvent:] + 284
9   AppKit                              0x00000001b2d8825c -[NSApplication(NSEvent) sendEve

The rest of this code were previous attempts that don't work

In [ ]:
# Don't use this, it uses an abritary cut-off of 125e-6 and 
# averages the latencies of all the peaks instead of just the first one

import mne
import numpy as np
import matplotlib.pyplot as plt

# Assuming 'raw' and 'event_id' are already defined and loaded

# Update the epoch parameters for the new time window and amplitude threshold
tmin_updated, tmax_updated = 0, 0.250
amplitude_threshold = 125e-6  # 125 µV in Volts
events, event_id = mne.events_from_annotations(raw)
stimulus_s1_events = events[events[:, 2] == event_id['Stimulus/s1']]

# Recreate epochs for Stimulus/s1 within the specified window
epochs_updated = mne.Epochs(raw, events=stimulus_s1_events, event_id=event_id['Stimulus/s1'],
                            tmin=tmin_updated, tmax=tmax_updated, preload=True, baseline=None)

# Pick the BIP3 channel
epochs_updated = epochs_updated.pick_channels(['BIP3'])

# Find the first timepoint exceeding 125 µV after the stimulus in each epoch
exceeding_threshold_latencies_updated = []
for epoch in epochs_updated.get_data():
    # Find the index where the amplitude first exceeds 125 µV
    exceeding_index_updated = np.where(epoch[0, :] > amplitude_threshold)[0]
    if exceeding_index_updated.size > 0:  # Check if there's at least one exceeding point
        first_exceeding_time_updated = epochs_updated.times[exceeding_index_updated[0]]
        exceeding_threshold_latencies_updated.append(first_exceeding_time_updated)
    else:
        exceeding_threshold_latencies_updated.append(None)

# Filter out None values for calculating average and visualization
valid_latencies_updated = [latency for latency in exceeding_threshold_latencies_updated if latency is not None]

# Calculate the average latency of valid latencies
average_latency_threshold_updated = np.median(valid_latencies_updated) if valid_latencies_updated else 0

print(f"Number of valid latencies: {len(valid_latencies_updated)}, Average latency: {average_latency_threshold_updated}")

# Visualize the distribution of valid latencies
plt.figure(figsize=(10, 6))
plt.hist(valid_latencies_updated, bins=20, color='orange', edgecolor='black')
plt.axvline(average_latency_threshold_updated, color='blue', linestyle='dashed', linewidth=2)
plt.title('Distribution of Photodiode Peaks >125 uV (0, 0.250)')
plt.xlabel('Latency (seconds)')
plt.ylabel('Frequency')
plt.legend(['Average Latency', 'Latencies'])
plt.grid(True)

plt.show()


In [149]:
# The heat map looks weird, not sure why...

epochs_time_windowed = mne.Epochs(raw, events=stimulus_s1_events, event_id=event_id['Stimulus/s1'],
                                  tmin=0, tmax=0.250, preload=True, baseline=(None, None))

epochs_time_windowed.plot_image(picks=['BIP3'], sigma=1.0, cmap='viridis', 
                          vmin=-150, vmax=150)


Not setting metadata
100 matching events found
Setting baseline interval to [0.0, 0.25] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 100 events and 126 original time points ...
0 bad epochs dropped
Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated


/var/folders/dv/s4k5_3r51ml3q36bgbcdsgbr0000gn/T/ipykernel_45409/3220100308.py:4: RuntimeWarning: Cannot find channel coordinates in the supplied Evokeds. Not showing channel locations.
  epochs_time_windowed.plot_image(picks=['BIP3'], sigma=1.0, cmap='viridis',


[<Figure size 640x480 with 3 Axes>]